# Load Parquet Files into DataFrames

In [15]:
import pandas as pd
import os
# Directory containing Parquet files
parquet_directory = 'ExtractedFiles'

# Initialize a dictionary to hold the DataFrames
dataframes = {}

# Check if the directory exists
if os.path.exists(parquet_directory):
    # List all Parquet files in the directory
    parquet_files = [f for f in os.listdir(parquet_directory) if f.endswith('.parquet')]

    # Process each Parquet file
    for parquet_file in parquet_files:
        filepath = os.path.join(parquet_directory, parquet_file)
        # Load the Parquet file into a DataFrame
        df = pd.read_parquet(filepath)
        
        # Add the DataFrame to the dictionary
        # Use the filename (without the extension) as the key
        key = os.path.splitext(parquet_file)[0]
        dataframes[key] = df

    print("Loaded DataFrames from Parquet files.")
else:
    print(f"Directory does not exist: {parquet_directory}")

# Iterate through each key, value pair in the dataframes dictionary
for key, df in dataframes.items():
    print(f"Information for DataFrame: {key}")
    # Use the info() method to print detailed information about the DataFrame
    df.info()
    #df.head()
    print("\n")  # Add a newline for better readability between DataFrame infos

# Iterate through each key, value pair in the dataframes dictionary
for key, df in dataframes.items():
    print(f"Head of DataFrame '{key}':")
    print(df.head())  # Print the first few rows of the DataFrame
    print("-" * 40)  # Print a separator for readability


Loaded DataFrames from Parquet files.
Information for DataFrame: activity
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20472180 entries, 0 to 20472179
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   EntityNumber    object
 1   ActivityGroup   int64 
 2   NaceVersion     int64 
 3   NaceCode        int64 
 4   Classification  object
dtypes: int64(3), object(2)
memory usage: 781.0+ MB


Information for DataFrame: address
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2821223 entries, 0 to 2821222
Data columns (total 13 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   EntityNumber      object
 1   TypeOfAddress     object
 2   CountryNL         object
 3   CountryFR         object
 4   Zipcode           object
 5   MunicipalityNL    object
 6   MunicipalityFR    object
 7   StreetNL          object
 8   StreetFR          object
 9   HouseNumber       object
 10  Box               object
 11  ExtraAddressInfo  o

# Filtered DataFrame

In [59]:
import pandas as pd
import os
from functools import reduce

# Filtering for 'OfficialName'
official_name_df = dataframes['denomination'][
    (dataframes['denomination']['TypeOfDenomination'] == 1) & 
    (dataframes['denomination']['Language'] == 2)
][['Denomination', 'EntityNumber']].copy()  # Assuming 'Denomination' is the correct column to use

# Renaming column as per requirement
official_name_df.rename(columns={'Denomination': 'OfficialName'}, inplace=True)


abbreviation_df = dataframes['denomination'][
    (dataframes['denomination']['TypeOfDenomination'] == 2) & 
    (dataframes['denomination']['Language'] == 2)
][['Denomination', 'EntityNumber']].copy()  


abbreviation_df.rename(columns={'Denomination': 'Abbreviation'}, inplace=True)

tradeName_df = dataframes['denomination'][
    (dataframes['denomination']['TypeOfDenomination'] == 3) & 
    (dataframes['denomination']['Language'] == 2)
][['Denomination', 'EntityNumber']].copy()  

tradeName_df.rename(columns={'Denomination': 'TradeName'}, inplace=True)


zipcode_df = dataframes['address'][
    (dataframes['address']['TypeOfAddress'] == "REGO")
][['Zipcode', 'EntityNumber']].copy()  

zipcode_df.rename(columns={'Zipcode': 'ZipCode'}, inplace=True)


Municipality_df = dataframes['address'][
    (dataframes['address']['TypeOfAddress'] == "REGO")
][['MunicipalityNL', 'EntityNumber']].copy()  

Municipality_df.rename(columns={'MunicipalityNL': 'Municipality'}, inplace=True)


Street_df = dataframes['address'][
    (dataframes['address']['TypeOfAddress'] == "REGO")
][['StreetNL', 'EntityNumber']].copy()  

Street_df.rename(columns={'StreetNL': 'Street'}, inplace=True)


HouseNumber_df = dataframes['address'][
    (dataframes['address']['TypeOfAddress'] == "REGO")
][['HouseNumber', 'EntityNumber']].copy()  

HouseNumber_df.rename(columns={'HouseNumber': 'HouseNumber'}, inplace=True)

Email_df = dataframes['contact'][
    (dataframes['contact']['EntityContact'].isin(['ENT', 'EST'])) & 
    (dataframes['contact']['ContactType'] == 'EMAIL')
][['Value', 'EntityNumber']].copy()

Email_df.rename(columns={'Value': 'Email'}, inplace=True)

Tel_df = dataframes['contact'][
    (dataframes['contact']['EntityContact'].isin(['ENT', 'EST'])) & 
    (dataframes['contact']['ContactType'] == 'TEL')
][['Value', 'EntityNumber']].copy()

Tel_df.rename(columns={'Value': 'TEL'}, inplace=True)

URL_df = dataframes['contact'][
    (dataframes['contact']['EntityContact'].isin(['ENT', 'EST'])) & 
    (dataframes['contact']['ContactType'] == 'WEB')
][['Value', 'EntityNumber']].copy()

URL_df.rename(columns={'Value': 'URL'}, inplace=True)


# Combining the filtered DataFrames
# Assuming df_list is a list of your DataFrames
df_list = [official_name_df, abbreviation_df, tradeName_df,zipcode_df, Municipality_df, Street_df, HouseNumber_df, Email_df, Tel_df, URL_df]

# Using reduce to merge all DataFrames in the list based on 'EntityNumber'
combined_df = reduce(lambda left, right: pd.merge(left, right, on='EntityNumber', how='outer'), df_list)

# Now, 'EntityNumber' can be set as the index in the combined DataFrame
combined_df.set_index('EntityNumber', inplace=True)



# Filtering

In [60]:
# Step 1: Identify rows in 'address' DataFrame with missing 'CountryNL' values
missing_country_nl = dataframes['address']['CountryNL'].isna()

# Extract 'EntityNumber' for rows where 'CountryNL' is missing
missing_entities = dataframes['address'][missing_country_nl]['EntityNumber']

# Step 2: Filter these 'EntityNumber's out of your main dataset 'final_df'
# Assuming 'final_df' has 'EntityNumber' as its index or a column
if 'EntityNumber' in combined_df.columns:
    # If 'EntityNumber' is a column in 'final_df'
    final_df_filtered = combined_df[~combined_df['EntityNumber'].isin(missing_entities)]
else:
    # If 'EntityNumber' is the index in 'final_df'
    final_df_filtered = combined_df[~combined_df.index.isin(missing_entities)]

# Saving the combined dataset

In [61]:
# Define the path for the new combined Parquet file in your directory
new_combined_parquet_file_path = './combined_filtered_dataset.parquet'

# Save the combined DataFrame as a Parquet file
combined_df.to_parquet(new_combined_parquet_file_path)

print(f"Combined and filtered DataFrame saved as Parquet file at: {new_combined_parquet_file_path}")

Combined and filtered DataFrame saved as Parquet file at: ./combined_filtered_dataset.parquet


In [66]:
print(combined_df.info())
print(combined_df.head(50))

<class 'pandas.core.frame.DataFrame'>
Index: 2085477 entries, 0200.065.765 to 2.356.182.359
Data columns (total 10 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   OfficialName  object
 1   Abbreviation  object
 2   TradeName     object
 3   ZipCode       object
 4   Municipality  object
 5   Street        object
 6   HouseNumber   object
 7   Email         object
 8   TEL           object
 9   URL           object
dtypes: object(10)
memory usage: 239.5+ MB
None
                                                   OfficialName Abbreviation  \
EntityNumber                                                                   
0200.065.765               Intergemeentelijke Vereniging Veneco       Veneco   
0200.068.636                                              Farys          NaN   
0200.245.711               Intercommunaal Sanatorium Denderoord   DENDEROORD   
0200.305.493  Intergemeentelijk Samenwerkingsverband voor ru...        SOLVA   
0200.420.608      Congregatie der G